Integrating the SceneGraph notebooks with the Toybot notebooks

This includes code to run the bot as script.

SceneGraph
==

The scenegraph stores operations as nodes.

These include that change state (fill, stroke etc) and operations that result in elements being drawn (path)

In [1]:
class SceneGraph:
    def __init__(self):
        self.nodes = []
        
    def add_node(self, node):
        self.nodes.append(node)
    
    def __iter__(self):
        """
        Iterator that yields every node in the graph
        """
        for node in self.nodes:
            yield node

ConsoleRenderer
==

The ConsoleRenderer outputs every node of scenegraph to the console

In [2]:
class ConsoleRenderer:
    def render(self, graph):
        for node in graph:
            print(node)     

Grammar
==

The ToyBot class provides the user-facing API for drawing and setting up colours.

In [3]:
class ToyBot:
    def __init__(self, graph):
        self.graph = graph
    
    def rect(self, x, y, width, height, fill=None, stroke=None):
        coords = [(x, y), (x, y+height), (x+width, y+height), (x+width, y)]
        node = {'type': 'path', 'coords': coords}
        
        self.graph.add_node(node)

# Scripting

## Setup the scripting namespace

This function adds all the user-facing API of a bot into a namespace to enable scripting.

In [4]:
import inspect

def create_scripting_namespace(bot):
    namespace = {}
    for name, method in inspect.getmembers(bot, predicate=inspect.ismethod):
        if name.startswith('__'):
            continue
        namespace[name] = method
    return namespace

**Note:** This is only gathering functions, not variables.

## Running a script

The run function accepts a bot script, does the setup needed to render a bot and then renders.

In [5]:
def run(source):
    graph = SceneGraph()
    renderer = ConsoleRenderer()

    bot = ToyBot(graph)
    namespace = create_scripting_namespace(bot)
    
    exec(source, namespace)

    renderer.render(graph)

## Putting it all together - using the scripting interface

The scripting interface provides the simplest way of using ToyBot.

Pass the code to render to run function.

In [6]:
code = """
rect(0, 0, 100, 100)
rect(25, 30, 50, 70)
"""

run(code)

{'type': 'path', 'coords': [(0, 0), (0, 100), (100, 100), (100, 0)]}
{'type': 'path', 'coords': [(25, 30), (25, 100), (75, 100), (75, 30)]}
